# Bibliotecas

In [57]:
import pandas as pd
import numpy as np
import glob
import os

# Carregamento por Arquivo CSV

In [58]:
arquivo_csv = ('dataclean/processos_6.csv')
df = pd.read_csv(arquivo_csv, sep='#', encoding='utf-8')
print(df.shape)

C:\Users\jcpsrodrigues\AppData\Local\Temp\ipykernel_18212\2895580906.py:2: DtypeWarning: Columns (5,12,14,15,18,20,21,22,23,24,25,43,44,49,50,54,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_csv, sep='#', encoding='utf-8')


(2020987, 66)


# Validações Rápidas

In [135]:
pendentes = df[
    (df['serventia'] == '1º Juizado Especial Criminal') & 
    (df['comarca'] == 'APARECIDA DE GOIÂNIA')
]['data_baixa'].isna().sum()

print(f"Quantidade de valores nulos em 'data_baixa': {pendentes}")

Quantidade de valores nulos em 'data_baixa': 1286


In [136]:
baixados = df[
    (df['serventia'] == '1º Juizado Especial Criminal') & 
    (df['comarca'] == 'APARECIDA DE GOIÂNIA')
]['data_baixa'].notna().sum()

print(f"Quantidade de valores não-nulos em 'data_baixa': {baixados}")

Quantidade de valores não-nulos em 'data_baixa': 2454


# Limpeza e Tratamento

In [59]:
# Verificar o nome correto das colunas (pode haver diferenças de acentuação ou espaços)
colunas = df.columns.tolist()

# Encontrar as colunas de data corretamente
coluna_serventia = [col for col in colunas if 'serventia' in col.lower()][0]
coluna_distribuicao = [col for col in colunas if 'data_distribuicao' in col.lower()][0]
coluna_baixa = [col for col in colunas if 'data_baixa' in col.lower()][0]
coluna_area_acao = [col for col in colunas if 'nome_area_acao' in col.lower()][0]
coluna_processo_id = [col for col in colunas if 'processo_id' in col.lower()][0]
coluna_comarca = [col for col in colunas if 'comarca' in col.lower()][0]

# Renomear colunas para garantir consistência
df = df.rename(columns={
coluna_distribuicao: 'data_distribuicao',
coluna_baixa: 'data_baixa',
coluna_area_acao: 'nome_area_acao',
coluna_processo_id: 'processo_id',
coluna_comarca: 'comarca',
coluna_serventia: 'serventia'
})

# ==========================================================================================================================================
# ANÁLISE ANUAL
# ==========================================================================================================================================

In [93]:
# Gerando o Dataframe com a Taxa de Congestionamento Anual:
# 0) Garantir datetime
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

keys = ['comarca', 'serventia'] # Altere aqui para fazer os agrupamentos

# 1) Cópia 
df_f = df.copy()

# 2.1) Referência anual
df_f['ano_dist'] = df_f['data_distribuicao'].dt.to_period('Y')
df_f['ano_baixa'] = df_f['data_baixa'].dt.to_period('Y')

# 3.1) Fluxos anuais
# Distribuídos por ano (fluxo)
dist_ano_base = (
    df_f.groupby(keys + ['ano_dist'])
        .size()
        .rename('Distribuidos_fluxo_anual')
        .reset_index()
        .rename(columns={'ano_dist': 'ano_ref'})
)

# Baixados por ano (fluxo)
baix_ano_base = (
    df_f.dropna(subset=['data_baixa'])
        .groupby(keys + ['ano_baixa'])
        .size()
        .rename('Baixados_fluxo_anual')
        .reset_index()
        .rename(columns={'ano_baixa': 'ano_ref'})
)

# Calcular valores acumulados
# Distribuídos acumulados
dist_acum = dist_ano_base.copy()
dist_acum = dist_acum.sort_values(['comarca', 'serventia', 'ano_ref'])
dist_acum['Distribuidos_acum_ano'] = dist_acum.groupby(keys)['Distribuidos_fluxo_anual'].cumsum()

# Baixados acumulados
baix_acum = baix_ano_base.copy()
baix_acum = baix_acum.sort_values(['comarca', 'serventia', 'ano_ref'])
baix_acum['Baixados_acum_ano'] = baix_acum.groupby(keys)['Baixados_fluxo_anual'].cumsum()

# Pendentes acumulados = Distribuídos acumulados - Baixados acumulados
pend_acum = dist_acum[keys + ['ano_ref', 'Distribuidos_acum_ano']].copy()
pend_acum = pend_acum.merge(
    baix_acum[keys + ['ano_ref', 'Baixados_acum_ano']], 
    on=keys + ['ano_ref'], 
    how='left'
)
pend_acum['Baixados_acum_ano'] = pend_acum['Baixados_acum_ano'].fillna(0).astype(int)
pend_acum['Pendentes_acum_ano'] = pend_acum['Distribuidos_acum_ano'] - pend_acum['Baixados_acum_ano']

# 6.1) Dataframe anual (APENAS COM AS COLUNAS SOLICITADAS)
df_global_anual = dist_ano_base[['comarca', 'serventia', 'ano_ref', 'Distribuidos_fluxo_anual']].copy()

# Adicionar Baixados_fluxo_anual
df_global_anual = df_global_anual.merge(
    baix_ano_base[['comarca', 'serventia', 'ano_ref', 'Baixados_fluxo_anual']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Adicionar Pendentes_acum_ano
df_global_anual = df_global_anual.merge(
    pend_acum[['comarca', 'serventia', 'ano_ref', 'Pendentes_acum_ano']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Adicionar Distribuidos_acum_ano (apenas para cálculo, depois removemos)
df_global_anual = df_global_anual.merge(
    dist_acum[['comarca', 'serventia', 'ano_ref', 'Distribuidos_acum_ano']],
    on=['comarca', 'serventia', 'ano_ref'],
    how='left'
)

# Preencher NaN com 0 e Converter para int
df_global_anual['Baixados_fluxo_anual'] = df_global_anual['Baixados_fluxo_anual'].fillna(0).astype(int)
df_global_anual['Pendentes_acum_ano'] = df_global_anual['Pendentes_acum_ano'].fillna(0).astype(int)
df_global_anual['Distribuidos_fluxo_anual'] = df_global_anual['Distribuidos_fluxo_anual'].astype(int)
df_global_anual['Distribuidos_acum_ano'] = df_global_anual['Distribuidos_acum_ano'].astype(int)

# Renomear colunas EXATAMENTE como solicitado
df_global_anual = df_global_anual.rename(columns={
    'Distribuidos_fluxo_anual': 'Distribuidos_ano',
    'Baixados_fluxo_anual': 'Baixados_ano',
    'Pendentes_acum_ano': 'Pendentes_ano'
})

# Calcular taxa anual CORRETA: Pendentes_ano / Distribuidos_acum_ano
df_global_anual['Taxa_Cong_anual (%)'] = np.where(
    df_global_anual['Distribuidos_acum_ano'] > 0,
    (df_global_anual['Pendentes_ano'] / df_global_anual['Distribuidos_acum_ano']) * 100,
    0
).round(2)

# Remover a coluna auxiliar Distribuidos_acum_ano
df_global_anual = df_global_anual.drop('Distribuidos_acum_ano', axis=1)

# Ordenar por comarca, serventia e ano
df_global_anual = df_global_anual.sort_values(['comarca', 'serventia', 'ano_ref'])

In [95]:
# Validação mais refinada
ano_alvo = 2025
df_global_anual[
    (df_global_anual['comarca'] == 'APARECIDA DE GOIÂNIA') & 
    (df_global_anual['serventia'] == '1º Juizado Especial Criminal') &
    (df_global_anual['ano_ref'].dt.year == ano_alvo)
].head()

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
743,APARECIDA DE GOIÂNIA,1º Juizado Especial Criminal,2025,1009,1598,837,34.37


In [97]:
# Filtrando o ANO para exração ANUAL
ano_alvo = 2025
df_ano = df_global_anual[df_global_anual['ano_ref'].dt.year == ano_alvo]
df_ano.head()

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
26,ABADIÂNIA,Vara Judicial,2025,1938,2270,2781,55.06
61,ALEXÂNIA,Vara Judicial,2025,3827,5112,5957,53.82
97,ALVORADA DO NORTE,Vara Judicial,2025,2170,2501,4272,63.07
126,ANÁPOLIS,1ª Vara Criminal,2025,1035,1756,2198,55.59
162,ANÁPOLIS,1ª Vara Cível,2025,2369,2061,4857,70.21


In [98]:
# Grava cada arquivo ANUAL na pasta results
ano_alvo = 2025
df_ano = df_global_anual[df_global_anual['ano_ref'].dt.year == ano_alvo]
os.makedirs("results", exist_ok=True)
df_ano.to_csv(f"results/tx_{ano_alvo}.csv", sep=",", index=False, encoding="utf-8-sig")

In [99]:
# Concatena os arquivos ANUAIS da pasta results
# Listar os arquivos CSV na pasta 'results'
arquivo_csv = glob.glob('results/tx_*.csv')
# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
for arquivo in arquivo_csv:  # lista/iterável com os caminhos tipo 'tx_2020.csv', 'tx_2021.csv', ...
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    dfs.append(df_ano)

df_results_anual = pd.concat(dfs, ignore_index=True)

print(df_results_anual.shape)

(1998, 7)


In [102]:
# Validação Rápida do arquivo ANUAL concatenado
df_results_anual.head()

,comarca,serventia,ano_ref,Distribuidos_ano,Baixados_ano,Pendentes_ano,Taxa_Cong_anual (%)
0,ABADIÂNIA,Vara Judicial,2020,1365,1262,3841,75.27
1,ALEXÂNIA,Vara Judicial,2020,2489,2685,6577,71.01
2,ALVORADA DO NORTE,Vara Judicial,2020,1365,1477,6032,80.33
3,ANÁPOLIS,1ª Vara Criminal,2020,1161,1079,4378,80.23
4,ANÁPOLIS,1ª Vara Cível,2020,916,1645,4836,74.62


In [103]:
# Grava o arquivo ANUAL concatenado, no formato JSON, na pasta results_concat
os.makedirs("results_concat", exist_ok=True)
df_results_anual.to_json("results_concat/tx_cong_anual.json", orient="records", force_ascii=False)

# ==========================================================================================================================================
# ANÁLISE MENSAL
# ==========================================================================================================================================

In [149]:
# Análise Mensal
# Gerando o Dataframe com a Taxa de Congestionamento Mensal:
# 0) Garantir datetime
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

keys = ['comarca', 'serventia'] # Altere aqui para fazer os agrupamentos

# 1) Cópia 
df_f = df.copy()

# 2.1) Referência mensal
df_f['mes_dist'] = df_f['data_distribuicao'].dt.to_period('M')
df_f['mes_baixa'] = df_f['data_baixa'].dt.to_period('M')

# 3.1) Fluxos mensais
# Distribuídos por mês (fluxo)
dist_mes_base = (
    df_f.groupby(keys + ['mes_dist'])
        .size()
        .rename('Distribuidos_fluxo_mensal')
        .reset_index()
        .rename(columns={'mes_dist': 'mes_ref'})
)

# Baixados por mês (fluxo)
baix_mes_base = (
    df_f.dropna(subset=['data_baixa'])
        .groupby(keys + ['mes_baixa'])
        .size()
        .rename('Baixados_fluxo_mensal')
        .reset_index()
        .rename(columns={'mes_baixa': 'mes_ref'})
)

# Calcular valores acumulados
# Distribuídos acumulados
dist_acum = dist_mes_base.copy()
dist_acum = dist_acum.sort_values(['comarca', 'serventia', 'mes_ref'])
dist_acum['Distribuidos_acum_mes'] = dist_acum.groupby(keys)['Distribuidos_fluxo_mensal'].cumsum()

# Baixados acumulados
baix_acum = baix_mes_base.copy()
baix_acum = baix_acum.sort_values(['comarca', 'serventia', 'mes_ref'])
baix_acum['Baixados_acum_mes'] = baix_acum.groupby(keys)['Baixados_fluxo_mensal'].cumsum()

# Pendentes acumulados = Distribuídos acumulados - Baixados acumulados
pend_acum = dist_acum[keys + ['mes_ref', 'Distribuidos_acum_mes']].copy()
pend_acum = pend_acum.merge(
    baix_acum[keys + ['mes_ref', 'Baixados_acum_mes']], 
    on=keys + ['mes_ref'], 
    how='left'
)
pend_acum['Baixados_acum_mes'] = pend_acum['Baixados_acum_mes'].fillna(0).astype(int)
pend_acum['Pendentes_acum_mes'] = pend_acum['Distribuidos_acum_mes'] - pend_acum['Baixados_acum_mes']

# 6.1) Dataframe mensal
df_global_mensal = dist_mes_base[['comarca', 'serventia', 'mes_ref', 'Distribuidos_fluxo_mensal']].copy()

# Adicionar Baixados_fluxo_mensal
df_global_mensal = df_global_mensal.merge(
    baix_mes_base[['comarca', 'serventia', 'mes_ref', 'Baixados_fluxo_mensal']],
    on=['comarca', 'serventia', 'mes_ref'],
    how='left'
)

# Adicionar Pendentes_acum_mes
df_global_mensal = df_global_mensal.merge(
    pend_acum[['comarca', 'serventia', 'mes_ref', 'Pendentes_acum_mes']],
    on=['comarca', 'serventia', 'mes_ref'],
    how='left'
)

# Adicionar Distribuidos_acum_mes (apenas para cálculo, depois removemos)
df_global_mensal = df_global_mensal.merge(
    dist_acum[['comarca', 'serventia', 'mes_ref', 'Distribuidos_acum_mes']],
    on=['comarca', 'serventia', 'mes_ref'],
    how='left'
)

# Preencher NaN com 0 e Converter para int
df_global_mensal['Baixados_fluxo_mensal'] = df_global_mensal['Baixados_fluxo_mensal'].fillna(0).astype(int)
df_global_mensal['Pendentes_acum_mes'] = df_global_mensal['Pendentes_acum_mes'].fillna(0).astype(int)
df_global_mensal['Distribuidos_fluxo_mensal'] = df_global_mensal['Distribuidos_fluxo_mensal'].astype(int)
df_global_mensal['Distribuidos_acum_mes'] = df_global_mensal['Distribuidos_acum_mes'].astype(int)

# Renomear colunas
df_global_mensal = df_global_mensal.rename(columns={
    'Distribuidos_fluxo_mensal': 'Distribuidos_mes',
    'Baixados_fluxo_mensal': 'Baixados_mes',
    'Pendentes_acum_mes': 'Pendentes_mes'
})

# Calcular taxa mensal CORRETA: Pendentes_mes / Distribuidos_acum_mes
df_global_mensal['Taxa_Cong_mensal (%)'] = np.where(
    df_global_mensal['Distribuidos_acum_mes'] > 0,
    (df_global_mensal['Pendentes_mes'] / df_global_mensal['Distribuidos_acum_mes']) * 100,
    0
).round(2)

# Remover a coluna auxiliar Distribuidos_acum_mes
df_global_mensal = df_global_mensal.drop('Distribuidos_acum_mes', axis=1)

# Ordenar por comarca, serventia e mês
df_global_mensal = df_global_mensal.sort_values(['comarca', 'serventia', 'mes_ref'])

In [ ]:
# Filtrando todos os MESES de cada ano para exração MENSAL
ano_alvo = 2025
df_mes = df_global_mensal[df_global_mensal['mes_ref'].dt.year == ano_alvo]
df_mes.head(12)

,comarca,serventia,mes_ref,Distribuidos_mes,Baixados_mes,Pendentes_mes,Taxa_Cong_mensal (%)
208,ABADIÂNIA,Vara Judicial,2025-01,142,89,3166,97.27
209,ABADIÂNIA,Vara Judicial,2025-02,148,231,3083,90.60
210,ABADIÂNIA,Vara Judicial,2025-03,129,207,3005,85.08
211,ABADIÂNIA,Vara Judicial,2025-04,170,155,3020,81.58
212,ABADIÂNIA,Vara Judicial,2025-05,188,256,2952,75.89


In [151]:
# Grava cada arquivo MENSAL na pasta results
ano_alvo = 2025
df_mes = df_global_mensal[df_global_mensal['mes_ref'].dt.year == ano_alvo]
os.makedirs("results", exist_ok=True)
df_mes.to_csv(f"results/txmes_{ano_alvo}.csv", sep=",", index=False, encoding="utf-8-sig")

In [152]:
# Concatena os arquivos MENSAIS da pasta results
# Listar os arquivos CSV na pasta 'results'
arquivo_csv = glob.glob('results/txmes_*.csv')
# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
for arquivo in arquivo_csv:  # lista/iterável com os caminhos tipo 'txmes_2020.csv', 'txmes_2021.csv', ...
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    dfs.append(df_ano)

df_results_mensal = pd.concat(dfs, ignore_index=True)

print(df_results_mensal.shape)

(23448, 7)


In [155]:
# Validação Rápida do arquivo MENSAL concatenado
df_results_mensal.head()

,comarca,serventia,mes_ref,Distribuidos_mes,Baixados_mes,Pendentes_mes,Taxa_Cong_mensal (%)
0,ABADIÂNIA,Vara Judicial,2020-01,120,189,3669,95.10
1,ABADIÂNIA,Vara Judicial,2020-02,97,72,3694,93.40
2,ABADIÂNIA,Vara Judicial,2020-03,215,177,3732,89.50
3,ABADIÂNIA,Vara Judicial,2020-04,49,76,3705,87.82
4,ABADIÂNIA,Vara Judicial,2020-05,101,88,3718,86.06


In [157]:
filtro = (
    (df_results_mensal['comarca'] == 'ABADIÂNIA') & 
    (df_results_mensal['serventia'] == 'Vara Judicial') #&
    #(df_results_mensal['mes_ref'].astype(str).str.contains('2021'))
)
df_results_mensal[filtro].head()

,comarca,serventia,mes_ref,Distribuidos_mes,Baixados_mes,Pendentes_mes,Taxa_Cong_mensal (%)
0,ABADIÂNIA,Vara Judicial,2020-01,120,189,3669,95.10
1,ABADIÂNIA,Vara Judicial,2020-02,97,72,3694,93.40
2,ABADIÂNIA,Vara Judicial,2020-03,215,177,3732,89.50
3,ABADIÂNIA,Vara Judicial,2020-04,49,76,3705,87.82
4,ABADIÂNIA,Vara Judicial,2020-05,101,88,3718,86.06


In [158]:
# Grava o arquivo MENSAL concatenado, no formato JSON, na pasta results_concat
df_results_mensal.to_json("results_concat/tx_cong_mensal.json", orient="records", force_ascii=False)

# ==========================================================================================================================================
# ANÁLISE SEMANAL
# ==========================================================================================================================================

In [60]:
# Análise Semanal
# Gerando o Dataframe com a Taxa de Congestionamento Semanal:
# 0) Garantir datetime
df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
df['data_baixa'] = pd.to_datetime(df['data_baixa'], errors='coerce')

keys = ['comarca', 'serventia'] # Altere aqui para fazer os agrupamentos

# 1) Cópia 
df_f = df.copy()

# 2.1) Referência semanal
df_f['sem_dist'] = df_f['data_distribuicao'].dt.to_period('W')
df_f['sem_baixa'] = df_f['data_baixa'].dt.to_period('W')

# 3.1) Fluxos semanais
# Distribuídos por semana (fluxo)
dist_sem_base = (
    df_f.groupby(keys + ['sem_dist'])
        .size()
        .rename('Distribuidos_fluxo_semanal')
        .reset_index()
        .rename(columns={'sem_dist': 'sem_ref'})
)

# Baixados por semana (fluxo)
baix_sem_base = (
    df_f.dropna(subset=['data_baixa'])
        .groupby(keys + ['sem_baixa'])
        .size()
        .rename('Baixados_fluxo_semanal')
        .reset_index()
        .rename(columns={'sem_baixa': 'sem_ref'})
)

# Calcular valores acumulados
# Distribuídos acumulados
dist_acum = dist_sem_base.copy()
dist_acum = dist_acum.sort_values(['comarca', 'serventia', 'sem_ref'])
dist_acum['Distribuidos_acum_sem'] = dist_acum.groupby(keys)['Distribuidos_fluxo_semanal'].cumsum()

# Baixados acumulados
baix_acum = baix_sem_base.copy()
baix_acum = baix_acum.sort_values(['comarca', 'serventia', 'sem_ref'])
baix_acum['Baixados_acum_sem'] = baix_acum.groupby(keys)['Baixados_fluxo_semanal'].cumsum()

# Pendentes acumulados = Distribuídos acumulados - Baixados acumulados
pend_acum = dist_acum[keys + ['sem_ref', 'Distribuidos_acum_sem']].copy()
pend_acum = pend_acum.merge(
    baix_acum[keys + ['sem_ref', 'Baixados_acum_sem']], 
    on=keys + ['sem_ref'], 
    how='left'
)
pend_acum['Baixados_acum_sem'] = pend_acum['Baixados_acum_sem'].fillna(0).astype(int)
pend_acum['Pendentes_acum_sem'] = pend_acum['Distribuidos_acum_sem'] - pend_acum['Baixados_acum_sem']

# 6.1) Dataframe semanal
df_global_semanal = dist_sem_base[['comarca', 'serventia', 'sem_ref', 'Distribuidos_fluxo_semanal']].copy()

# Adicionar Baixados_fluxo_semanal
df_global_semanal = df_global_semanal.merge(
    baix_sem_base[['comarca', 'serventia', 'sem_ref', 'Baixados_fluxo_semanal']],
    on=['comarca', 'serventia', 'sem_ref'],
    how='left'
)

# Adicionar Pendentes_acum_sem
df_global_semanal = df_global_semanal.merge(
    pend_acum[['comarca', 'serventia', 'sem_ref', 'Pendentes_acum_sem']],
    on=['comarca', 'serventia', 'sem_ref'],
    how='left'
)

# Adicionar Distribuidos_acum_sem (apenas para cálculo, depois removemos)
df_global_semanal = df_global_semanal.merge(
    dist_acum[['comarca', 'serventia', 'sem_ref', 'Distribuidos_acum_sem']],
    on=['comarca', 'serventia', 'sem_ref'],
    how='left'
)

# Preencher NaN com 0 e Converter para int
df_global_semanal['Baixados_fluxo_semanal'] = df_global_semanal['Baixados_fluxo_semanal'].fillna(0).astype(int)
df_global_semanal['Pendentes_acum_sem'] = df_global_semanal['Pendentes_acum_sem'].fillna(0).astype(int)
df_global_semanal['Distribuidos_fluxo_semanal'] = df_global_semanal['Distribuidos_fluxo_semanal'].astype(int)
df_global_semanal['Distribuidos_acum_sem'] = df_global_semanal['Distribuidos_acum_sem'].astype(int)

# Renomear colunas
df_global_semanal = df_global_semanal.rename(columns={
    'Distribuidos_fluxo_semanal': 'Distribuidos_sem',
    'Baixados_fluxo_semanal': 'Baixados_sem',
    'Pendentes_acum_sem': 'Pendentes_sem'
})

# Calcular taxa semanal CORRETA: Pendentes_sem / Distribuidos_acum_sem
df_global_semanal['Taxa_Cong_semanal (%)'] = np.where(
    df_global_semanal['Distribuidos_acum_sem'] > 0,
    (df_global_semanal['Pendentes_sem'] / df_global_semanal['Distribuidos_acum_sem']) * 100,
    0
).round(2)

# Remover a coluna auxiliar Distribuidos_acum_sem
df_global_semanal = df_global_semanal.drop('Distribuidos_acum_sem', axis=1)

# Ordenar por comarca, serventia e semana
df_global_semanal = df_global_semanal.sort_values(['comarca', 'serventia', 'sem_ref'])

In [61]:
# Validação por SEMANA
ano_alvo = 2025
df_semanal= df_global_semanal[(df_global_semanal['sem_ref'].dt.year == ano_alvo)]

df_semanal.head()

,comarca,serventia,sem_ref,Distribuidos_sem,Baixados_sem,Pendentes_sem,Taxa_Cong_semanal (%)
566,ABADIÂNIA,Vara Judicial,2024-12-30/2025-01-05,9,0,3119,100.00
567,ABADIÂNIA,Vara Judicial,2025-01-06/2025-01-12,31,13,3137,99.59
568,ABADIÂNIA,Vara Judicial,2025-01-13/2025-01-19,21,9,3149,99.31
569,ABADIÂNIA,Vara Judicial,2025-01-20/2025-01-26,60,22,3187,98.64
570,ABADIÂNIA,Vara Judicial,2025-01-27/2025-02-02,25,45,3167,97.27


In [62]:
# Grava cada arquivo SEMANAL na pasta results
ano_alvo = 2025
df_semanal= df_global_semanal[(df_global_semanal['sem_ref'].dt.year == ano_alvo)]
df_semanal.to_csv(f"results/txsem_{ano_alvo}.csv", sep=",", index=False, encoding="utf-8-sig")

In [63]:
# Concatena os arquivos SEMANAIS da pasta results
# Listar os arquivos CSV na pasta 'results'
arquivo_csv = glob.glob('results/txsem_*.csv')
# Carregar os arquivos CSV e concatenar em um único DataFrame
dfs = []
for arquivo in arquivo_csv:  # lista/iterável com os caminhos tipo 'txsem_2020.csv', 'txsem_2021.csv', ...
    df_ano = pd.read_csv(arquivo, sep=',', encoding='utf-8')
    dfs.append(df_ano)

df_results_semanal = pd.concat(dfs, ignore_index=True)

print(df_results_semanal.shape)

(99740, 7)


In [64]:
# Validação Rápida do arquivo SEMANAL concatenado
df_results_semanal.head()

,comarca,serventia,sem_ref,Distribuidos_sem,Baixados_sem,Pendentes_sem,Taxa_Cong_semanal (%)
0,ABADIÂNIA,Vara Judicial,2019-12-30/2020-01-05,27,0,3739,100.00
1,ABADIÂNIA,Vara Judicial,2020-01-06/2020-01-12,28,6,3761,99.84
2,ABADIÂNIA,Vara Judicial,2020-01-13/2020-01-19,33,10,3784,99.58
3,ABADIÂNIA,Vara Judicial,2020-01-20/2020-01-26,26,139,3671,95.95
4,ABADIÂNIA,Vara Judicial,2020-01-27/2020-02-02,32,34,3669,95.10


In [65]:
# Grava o arquivo MENSAL concatenado, no formato JSON, na pasta results_concat
df_results_semanal.to_json("results_concat/tx_cong_semanal.json", orient="records", force_ascii=False)

In [14]:
# Filtrar apenas semanas de 2020
df_2020 = df_global_semanal[df_global_semanal['sem_ref'].astype(str).str.contains('2020')]

df_2020.head()

,comarca,serventia,sem_ref,Distribuidos_sem,Baixados_sem,Pendentes_sem,Taxa_Cong_semanal (%)
609,ABADIÂNIA,Vara Judicial,2019-12-30/2020-01-05,27,0,3739,100.00
610,ABADIÂNIA,Vara Judicial,2020-01-06/2020-01-12,28,6,3761,99.84
611,ABADIÂNIA,Vara Judicial,2020-01-13/2020-01-19,33,10,3784,99.58
612,ABADIÂNIA,Vara Judicial,2020-01-20/2020-01-26,26,139,3671,95.95
613,ABADIÂNIA,Vara Judicial,2020-01-27/2020-02-02,32,34,3669,95.10
